In [21]:
from pandas import read_csv, cut
from numpy import array, unique, log2, inf, append, where, square
from sklearn.model_selection import train_test_split
import random
from pprint import pprint

In [3]:
file_path = '../catalog1/cat1.csv'
df = read_csv(file_path)
df.head()
y = array(df['class'])
# df = df.rename(columns={"class": "label"})
if 'cat2.csv' in file_path:
    df.drop("Unnamed: 0.1", axis=1, inplace=True)
df.drop(["Unnamed: 0", "galex_objid", "sdss_objid", "class", "spectrometric_redshift", "pred"], axis=1, inplace=True)

In [4]:
def bucketize(dataframe, col_headers, bucket_size):
    assert len(col_headers) == len(bucket_size)
    no_of_columns = len(col_headers)
    for col in range(no_of_columns):
        labels = array([(x + 1) for x in range(bucket_size[col])])
        temp = cut(dataframe[col_headers[col]], bucket_size[col], labels=labels)
        dataframe.drop(col_headers[col], inplace=True, axis=1)
        dataframe[col_headers[col]] = temp
    return dataframe
temp = bucketize(df, df.columns, [4 for x in range(len(df.columns))])

In [5]:
X = array(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
def check_purity(y):
    if len(unique(y)) == 1:
        return True
    else:
        return False

In [7]:
def classify_data(y):
    unique_classes, counts_unique_classes = unique(y, return_counts=True)

    index = where(counts_unique_classes == max(counts_unique_classes))[0][0]
    classification = unique_classes[index]
    
    return classification

In [29]:
def get_potential_splits(X, y):
    
    potential_splits = {}
    #n_columns = len(X[0])
    for column_index in range(len(X[0])):  
        potential_splits[column_index] = []
        values = X[:, column_index]
        unique_values = unique(values)

        for index in range(1, len(unique_values)):
            current_value = unique_values[index]
            previous_value = unique_values[index - 1]
            potential_split = (current_value + previous_value) / 2

            potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [9]:
def split_data(X, y, split_column, split_value):
    
    no_of_columns = len(X[0]) + 1
    split_column_values = X[:, split_column]
    data = append(X, y.reshape(len(X), 1), axis=1)
#     data_below = data_above = array([]).reshape(0, no_of_columns)
#     print("len(X) in function split_data:", len(X))
#     for index in range(len(X)):
#         temp = array(append(X[index], y[index])).reshape(1, no_of_columns)
#         if split_column_values[index] <= split_value:
#             data_below = append(data_below, temp, axis=0)
#         else:
#             data_above = append(data_above, temp, axis=0)
    data_below = data[data[:, split_column] < split_value]
    data_above = data[data[:, split_column] >= split_value]
    
    return data_below, data_above

In [18]:
def calculate_entropy(label_column):
    
#     print(data)
    _, counts = unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
     
    return 1 - sum(square(probabilities))

In [19]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below[:, -1]) 
                      + p_data_above * calculate_entropy(data_above[:, -1]))
    
    return overall_entropy

In [12]:
def determine_best_split(X, y, potential_splits):    
    overall_entropy = inf
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(X, y, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [13]:
def decision_tree_algorithm(X, y, column_headers, max_depth, counter=0, min_samples=2):
   
        
    if (check_purity(y)) or (len(X) < min_samples) or (counter == max_depth):
        classification = classify_data(y)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(X, y)
        split_column, split_value = determine_best_split(X, y, potential_splits)
        data_below, data_above = split_data(X, y, split_column, split_value)
    
        feature_name = column_headers[split_column]
        question = "column_{} <= {}".format(split_column, split_value)
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below[:, :-1], data_below[:, -1], column_headers, max_depth, counter, min_samples)
        no_answer = decision_tree_algorithm(data_above[:, :-1], data_above[:, -1], column_headers, max_depth, counter, min_samples)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [30]:
import time
start_time = time.time()
tree = decision_tree_algorithm(X_train, y_train, column_headers=df.columns, max_depth=3)
end_time = time.time()
pprint(tree)
print("Time taken to construct the decision tree =", end_time - start_time)

{'column_23 <= 1.5': [{'column_28 <= 2.5': [1, 0]},
                      {'column_30 <= 1.5': [{'column_27 <= 2.5': [0, 1]}, 1]}]}
Time taken to construct the decision tree = 0.29285550117492676


In [15]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")
    x = int(feature_name.split("_")[1])
    
    # ask question
    if example[x] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [16]:
def predict(X_test, tree):
    predictions = array([])
    for example in X_test:
        predictions = append(predictions, classify_example(example, tree))
    
    return predictions

In [31]:
predictions = predict(X_test, tree)

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      0.61      0.76        18
           1       0.96      1.00      0.98       177

    accuracy                           0.96       195
   macro avg       0.98      0.81      0.87       195
weighted avg       0.97      0.96      0.96       195

